## Assignment 1. Neural Text Classification
## CS310 Natural Language Processing

**Total points**: 50

You should roughtly follow the structure of the notebook. Add additional cells if you feel needed. 

You can (and you should) re-use the code from Lab 2. 

Make sure your code is readable and well-structured.

### 0. Import Necessary Libraries

In [ ]:
import json
import re
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchtext.vocab import build_vocab_from_iterator
import time
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
import jieba
import paddle
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

### 1. Data Processing

In [11]:
class myDataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        processed_data = []
        for line in lines:
            json_data = json.loads(line)
            processed_data.append(json_data)
        self.data = processed_data


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


def basic_tokenizer(sentence):
    tokens = re.findall(r'[\u4e00-\u9fff]', sentence)
    return tokens

def improved_tokenizer(sentence):
    chinese_pattern = re.compile(r'[\u4e00-\u9fff]')
    digit_pattern = re.compile(r'\d+')
    english_pattern = re.compile(r'[a-zA-Z]+')
    # 匹配除了中英文数字空格之外的特殊字符
    punctuation_pattern = re.compile(r'[^\u4e00-\u9fff\da-zA-Z\s]')
    tokens = re.findall(r'[\u4e00-\u9fff]|\d+|[a-zA-Z]+|[^\u4e00-\u9fff\da-zA-Z\s]', sentence)
    return tokens

# Example usage
train_dataset = myDataset('train.jsonl')

train_iterator = iter(train_dataset)




def yield_tokens(data_iter):
    for item in data_iter:
        yield improved_tokenizer(item['sentence'])

count = 0
for tokens in yield_tokens(train_iterator): # Use a new iterator
    print(tokens)
    count += 1
    if count > 7:
        break



vocab = build_vocab_from_iterator(yield_tokens(train_iterator), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Print the vocabulary size
print("Vocabulary size:", len(vocab))

# Print a few random tokens and their corresponding indices
for token in ['保', '说', '，', '小']:
    print(f"Token: {token}, Index: {vocab[token]}")


text_pipeline = lambda x: vocab(improved_tokenizer(x))
label_pipeline = lambda x: int(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, token_ids_list, offsets = [], [], [0]
    for item in batch:
        label_list.append(label_pipeline(item['label'][0]))
        token_ids = torch.tensor(text_pipeline('sentence'), dtype=torch.int64)
        token_ids_list.append(token_ids)
        offsets.append(token_ids.size(0))

    labels = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    token_ids = torch.cat(token_ids_list)

    return labels.to(device), token_ids.to(device), offsets.to(device)



train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=False,collate_fn=collate_batch)
count = 0
# Test the dataloader
for i, item in enumerate(train_dataloader):
    print(i)
    print(item)
    count += 1
    if count > 7:
        break



# Test the dataloader
for i, (labels, token_ids, offsets) in enumerate(train_dataloader):
    print(f"batch {i} label: {labels}")
    print(f"batch {i} text: {token_ids}")
    print(f"batch {i} offsets: {offsets}")
    if i == 0:
        break

# What does offsets mean?
print('Number of tokens: ', token_ids.size(0))
print('Number of examples in one batch: ', labels.size(0))
print('Example 1: ', token_ids[offsets[0]:offsets[1]])
print('Example 8: ', token_ids[offsets[7]:])



['卖', '油', '条', '小', '刘', '说', '：', '我', '说']
['保', '姆', '小', '张', '说', '：', '干', '啥', '子', '嘛', '？']
['卖', '油', '条', '小', '刘', '说', '：', '你', '看', '你', '往', '星', '空', '看', '月', '朦', '胧', '，', '鸟', '朦', '胧']
['卖', '油', '条', '小', '刘', '说', '：', '咱', '是', '不', '是', '歇', '一', '下', '这', '双', '，', '疲', '惫', '的', '双', '腿', '？']
['卖', '油', '条', '小', '刘', '说', '：', '快', '把', '我', '累', '死', '了']
['卖', '油', '条', '小', '刘', '说', '：', '我', '说', '亲', '爱', '的', '大', '姐', '你', '贵', '姓', '啊', '？']
['保', '姆', '小', '张', '说', '：', '我', '免', '贵', '姓', '张', '我', '叫', '张', '凤', '姑']
['卖', '油', '条', '小', '刘', '说', '：', '凤', '姑']
Vocabulary size: 2820
Token: 保, Index: 74
Token: 说, Index: 1
Token: ，, Index: 6
Token: 小, Index: 23
0
(tensor([0, 0, 1, 0, 0, 1, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 2, 3, 4, 5, 6, 7]))
1
(tensor([0, 0, 0, 1, 1, 0, 0, 1]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 2, 3, 4, 5, 6, 7]))
2
(tensor([0, 1, 0, 1, 0, 1, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor

### 2. Build the Model

In [12]:

class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(Model, self).__init__()

        self.embedding = nn.EmbeddingBag(vocab_size, embedding_dim, sparse=False)
        #指定两个隐藏层，每个隐藏层由nn.Linear和nn.ReLU激活函数组成。最后一层是线性层，输出num_classes个类别
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )


    def forward(self, token_ids, offsets):
        embedded = self.embedding(token_ids, offsets)
        output = self.fc(embedded)
        return output


vocab_size = len(vocab)
embedding_dim = 64
hidden_dim = 256
num_classes = 2 #0,1

model = Model(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)

model.eval()
with torch.no_grad():
    for i, (labels, token_ids, offsets) in enumerate(train_dataloader):
        output = model(token_ids, offsets)
        # print(f"batch {i} output: {output}")
        if i == 0:
            break



output size: torch.Size([8, 2])
output: tensor([[0.0500, 0.0507],
        [0.0500, 0.0507],
        [0.0500, 0.0507],
        [0.0500, 0.0507],
        [0.0500, 0.0507],
        [0.0500, 0.0507],
        [0.0500, 0.0507],
        [0.0500, 0.0507]])


### 3. Train and Evaluate

In [15]:

def train(model, dataloader, optimizer, criterion, epoch: int):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (labels, token_ids, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(token_ids, offsets)
        try:
            loss = criterion(output, labels)
        except Exception:
            print('Error in loss calculation')
            print('output: ', output.size())
            print('labels: ', labels.size())
            # print('token_ids: ', token_ids)
            # print('offsets: ', offsets)
            raise
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (output.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

# def evaluate(model, dataloader, criterion):
#     model.eval()
#     total_acc, total_count = 0, 0
#
#     with torch.no_grad():
#         for idx, (label, text, offsets) in enumerate(dataloader):
#             output = model(text, offsets)
#             loss = criterion(output, label)
#             total_acc += (output.argmax(1) == label).sum().item()
#             total_count += label.size(0)
#
#     return total_acc / total_count

def evaluate(model, dataloader, criterion):
    model.eval()
    total_acc, total_count = 0, 0
    y_true, y_pred = [], []

    with torch.no_grad():
        for idx, (labels, token_ids, offsets) in enumerate(dataloader):
            output = model(token_ids, offsets)
            loss = criterion(output, labels)
            total_acc += (output.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

            y_true.extend(labels.tolist())
            y_pred.extend(output.argmax(1).tolist())

    accuracy = total_acc / total_count
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return accuracy, precision, recall, f1


# Hyperparameters
EPOCHS = 10  # epoch
LR = 0.1  # learning rate
BATCH_SIZE = 8  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

# First, obtain some output and labels
model.eval()
with torch.no_grad():
    for i, (labels, token_ids, offsets) in enumerate(train_dataloader):
        output = model(token_ids, offsets)
        # print(f"batch {i} output: {output}")
        if i == 0:
            break

loss = criterion(output, labels)
print('loss:', loss)

criterion2 = torch.nn.CrossEntropyLoss(reduction='none')
loss2 = criterion2(output, labels)
print('loss non-reduced:', loss2)
print('mean of loss non-reduced:', torch.mean(loss2))

# Manually calculate the loss
probs = torch.exp(output[0,:]) / torch.exp(output[0,:]).sum()
loss3 = -torch.log(probs[labels[0]])
print('loss manually computed:', loss3)





# Prepare train, valid, and test data
train_dataset = myDataset('train.jsonl')
test_dataset = myDataset('test.jsonl')
# train_dataset = to_map_style_dataset(train_iter)
# test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
### Main Training Loop

# Run the training loop
total_accu = None
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    print('$$$$$$$$$$$$$$$$$')
    train(model, train_dataloader, optimizer, criterion, epoch)
    accuracy, precision, recall, f1 = evaluate(model, valid_dataloader, criterion)

    if total_accu is not None and total_accu > accuracy:
        scheduler.step()
    else:
        total_accu = accuracy

    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} | precision {:8.3f} | recall {:6.3f} |f1 {:5.3f} ".format(
            epoch, time.time() - epoch_start_time, accuracy,precision,recall,f1
        )
    )
    print("-" * 59)


# Save the model
torch.save(model.state_dict(), "text_classification_model.pth")
# accu_test = evaluate(model, valid_dataloader, criterion)
# print("test accuracy {:8.3f}".format(accu_test))

loss: tensor(0.7067)
loss non-reduced: tensor([0.2744, 1.4272, 0.2744, 0.2744, 0.2744, 1.4272, 0.2744, 1.4272])
mean of loss non-reduced: tensor(0.7067)
loss manually computed: tensor(0.2744)
$$$$$$$$$$$$$$$$$
| epoch   1 |   500/ 1506 batches | accuracy    0.692
| epoch   1 |  1000/ 1506 batches | accuracy    0.687
| epoch   1 |  1500/ 1506 batches | accuracy    0.698
-----------------------------------------------------------
| end of epoch   1 | time:  7.21s | valid accuracy    0.713 
-----------------------------------------------------------
$$$$$$$$$$$$$$$$$
| epoch   2 |   500/ 1506 batches | accuracy    0.698
| epoch   2 |  1000/ 1506 batches | accuracy    0.668
| epoch   2 |  1500/ 1506 batches | accuracy    0.694
-----------------------------------------------------------
| end of epoch   2 | time:  7.26s | valid accuracy    0.713 
-----------------------------------------------------------
$$$$$$$$$$$$$$$$$
| epoch   3 |   500/ 1506 batches | accuracy    0.706
| epoch   3 | 

### 4. Explore Word Segmentation

In [14]:
def jieba_tokenizer(sentence):
    tokens=[]
    seg_list = jieba.cut(sentence)
    for seg in seg_list:
        tokens.append(seg)
    return tokens

# Example usage
train_dataset = myDataset('train.jsonl')

train_iterator = iter(train_dataset)




def yield_tokens(data_iter):
    for item in data_iter:
        yield jieba_tokenizer(item['sentence'])

count = 0
for tokens in yield_tokens(train_iterator): # Use a new iterator
    print(tokens)
    count += 1
    if count > 7:
        break



vocab = build_vocab_from_iterator(yield_tokens(train_iterator), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Print the vocabulary size
print("Vocabulary size:", len(vocab))

# Print a few random tokens and their corresponding indices
for token in ['保', '说', '，', '小']:
    print(f"Token: {token}, Index: {vocab[token]}")


text_pipeline = lambda x: vocab(jieba_tokenizer(x))
label_pipeline = lambda x: int(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, token_ids_list, offsets = [], [], [0]
    for item in batch:
        label_list.append(label_pipeline(item['label'][0]))
        token_ids = torch.tensor(text_pipeline('sentence'), dtype=torch.int64)
        token_ids_list.append(token_ids)
        offsets.append(token_ids.size(0))

    labels = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    token_ids = torch.cat(token_ids_list)

    return labels.to(device), token_ids.to(device), offsets.to(device)



train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=False,collate_fn=collate_batch)
count = 0
# Test the dataloader
for i, item in enumerate(train_dataloader):
    print(i)
    print(item)
    count += 1
    if count > 7:
        break



# Test the dataloader
for i, (labels, token_ids, offsets) in enumerate(train_dataloader):
    print(f"batch {i} label: {labels}")
    print(f"batch {i} text: {token_ids}")
    print(f"batch {i} offsets: {offsets}")
    if i == 0:
        break

# What does offsets mean?
print('Number of tokens: ', token_ids.size(0))
print('Number of examples in one batch: ', labels.size(0))
print('Example 1: ', token_ids[offsets[0]:offsets[1]])
print('Example 8: ', token_ids[offsets[7]:])


#######################################


# Example usage
vocab_size = len(vocab)
embedding_dim = 64
hidden_dim = 512
num_classes = 2 #0,1

model = Model(vocab_size, embedding_dim, hidden_dim, num_classes).to(device)

model.eval()
with torch.no_grad():
    for i, (labels, token_ids, offsets) in enumerate(train_dataloader):
        output = model(token_ids, offsets)
        # print(f"batch {i} output: {output}")
        if i == 0:
            break

# Examine the output
print('output size:', output.size())
print('output:', output)


########################################################

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

# First, obtain some output and labels
model.eval()
with torch.no_grad():
    for i, (labels, token_ids, offsets) in enumerate(train_dataloader):
        output = model(token_ids, offsets)
        # print(f"batch {i} output: {output}")
        if i == 0:
            break

loss = criterion(output, labels)
print('loss:', loss)

criterion2 = torch.nn.CrossEntropyLoss(reduction='none')
loss2 = criterion2(output, labels)
print('loss non-reduced:', loss2)
print('mean of loss non-reduced:', torch.mean(loss2))

# Manually calculate the loss
probs = torch.exp(output[0,:]) / torch.exp(output[0,:]).sum()
loss3 = -torch.log(probs[labels[0]])
print('loss manually computed:', loss3)



total_accu = None
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    print('$$$$$$$$$$$$$$$$$')
    train(model, train_dataloader, optimizer, criterion, epoch)
    accuracy, precision, recall, f1 = evaluate(model, valid_dataloader, criterion)

    if total_accu is not None and total_accu > accuracy:
        scheduler.step()
    else:
        total_accu = accuracy

    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} | precision {:8.3f} | recall {:6.3f} |f1 {:5.3f} ".format(
            epoch, time.time() - epoch_start_time, accuracy,precision,recall,f1
        )
    )
    print("-" * 59)


# Save the model
torch.save(model.state_dict(), "text_classification_jieba_model.pth")
# accu_test = evaluate(model, valid_dataloader, criterion)
# print("test accuracy {:8.3f}".format(accu_test))



Building prefix dict from the default dictionary ...
DEBUG 2024-03-05 11:06:42,886 __init__.py:113] Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/2s/8cp0j_3x6ts54xdw2_dpwhrh0000gn/T/jieba.cache
DEBUG 2024-03-05 11:06:42,887 __init__.py:132] Loading model from cache /var/folders/2s/8cp0j_3x6ts54xdw2_dpwhrh0000gn/T/jieba.cache
Loading model cost 0.494 seconds.
DEBUG 2024-03-05 11:06:43,380 __init__.py:164] Loading model cost 0.494 seconds.
Prefix dict has been built successfully.
DEBUG 2024-03-05 11:06:43,381 __init__.py:166] Prefix dict has been built successfully.


['卖', '油条', '小', '刘说', '：', '我', '说']
['保姆', '小张', '说', '：', '干', '啥子', '嘛', '？']
['卖', '油条', '小', '刘说', '：', '你', '看', '你', '往', '星空', '看', '月', '朦胧', '，', '鸟', '朦胧']
['卖', '油条', '小', '刘说', '：', '咱', '是不是', '歇', '一下', '这', '双', '，', '疲惫', '的', '双腿', '？']
['卖', '油条', '小', '刘说', '：', '快', '把', '我', '累死', '了']
['卖', '油条', '小', '刘说', '：', '我', '说', '亲爱', '的', '大姐', '你', '贵姓', '啊', '？']
['保姆', '小张', '说', '：', '我免', '贵姓', '张', '我', '叫', '张凤姑']
['卖', '油条', '小', '刘说', '：', '凤姑']
Vocabulary size: 13844
Token: 保, Index: 4141
Token: 说, Index: 2
Token: ，, Index: 4
Token: 小, Index: 82
0
(tensor([0, 0, 1, 0, 0, 1, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 2, 3, 4, 5, 6, 7]))
1
(tensor([0, 0, 0, 1, 1, 0, 0, 1]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 2, 3, 4, 5, 6, 7]))
2
(tensor([0, 1, 0, 1, 0, 1, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 2, 3, 4, 5, 6, 7]))
3
(tensor([0, 0, 1, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 2, 3, 4, 5, 6, 7]))
